In [2]:
import turkishnlp
from turkishnlp import detector
import pandas as pd
from turkish.deasciifier import Deasciifier
import re
import preprocessor as p #tweet-preprocessor
from  spacy.lang.tr.stop_words import STOP_WORDS
from spacy.lang.tr import Turkish
import string
import numpy as np

In [3]:
#method to clean - lemmatize, normalize, remove stop words
nlp = Turkish()
stopwords = list(STOP_WORDS)

def clean(sentence):
    #filter using spacy
    tokens = nlp(sentence)
    tokens = [word.lemma_ for word in tokens]
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word not in stopwords and word not in string.punctuation]
    return ' '.join(tokens)

## Preprocessing for reviews dataset

In [14]:
reviewsfile = 'reviews.xlsx'
reviews_df = pd.read_excel(reviewsfile, sheet_name=0, header=0, index_col=False, keep_default_na=True)
reviews_df.Review[5]

'eski olmasina ragmen ununu ve izlendigi zamanki etkisini hala koruyan bir film jack nicholson harika filmin adi ise filme bu kadar uyabilir d'

In [15]:
#deasciify reviews dataset
for i in range (len(reviews_df.Review)):
    deasciifier = Deasciifier(reviews_df.Review[i])
    reviews_df.at[i, 'Review'] = deasciifier.convert_to_turkish()

reviews_df.Review[5]

'eski olmasına rağmen ününü ve izlendiği zamanki etkisini hala koruyan bir film jack nicholson harika filmin adı ise filme bu kadar uyabilir d'

In [17]:
#save the dataframe as csv file
reviews_df.to_csv('reviews_deasciified.csv', index=False)

In [ ]:
#spelling correction - not executed for now since it takes a long time
obj = detector.TurkishNLP()
obj.create_word_set()

count = len(reviews_df.Review)
for i in range (count):
    lwords = obj.list_words(reviews_df.Review[i])
    corrected_words = obj.auto_correct(lwords)
    corrected_string = " ".join(corrected_words)
    reviews_df.at[i, 'Review'] = corrected_string

In [17]:
### PART 2 ###
#read reviews dataset
reviewspath = '/Users/pinarayaz/Jupyter/NLP/data/reviews_deasciified.csv'
reviews_df = pd.read_csv(reviewspath)
reviews_df.head()

,Review,Rating,Sentiment
0,nicholson gene harika,4.0,positive
1,mükemmel derece kötü diyen arkadaşın sinema bi...,5.0,positive
2,mükemmel derecede kötü bi film hep biselerin o...,1.0,negative
3,nasıl beğendiğinizi anlamıyorum bu filmi filmd...,1.5,negative
4,ok harika bir film senaryo gereğinden fazla ol...,5.0,positive


In [19]:
count = len(reviews_df.Review)
for i in range (count):
    clean_text = clean(reviews_df.Review[i])
    reviews_df.at[i, 'Review'] = clean_text
reviews_df.head()

,Review,Rating,Sentiment
0,nicholson harika,4.0,positive
1,mükemmel derece kötü arkadaş sinema bilgi şüph...,5.0,positive
2,mükemmel derece kötü bi film biselerin ol bekl...,1.0,negative
3,beğen anla film film oyun senaryo zayıf film f...,1.5,negative
4,ok harika bir film senaryo fazla ol yüksek per...,5.0,positive


In [20]:
#save to csv
path = '/Users/pinarayaz/Jupyter/NLP/data/reviews_preprocessed.csv'
reviews_df.to_csv(path, index=False)

## Preprocessing for tweets dataset

In [15]:
traintweetsfile = 'train_tweets.xlsx'
traintweets_df = pd.read_excel(traintweetsfile, sheet_name=0, header=0, index_col=False, keep_default_na=True)
testtweetsfile = 'test_tweets.xlsx'
testtweets_df = pd.read_excel(testtweetsfile, sheet_name=0, header=0, index_col=False, keep_default_na=True)

tweets_df = pd.concat([traintweets_df, testtweets_df], axis=0, ignore_index=True)
tweets_df.Tweet[17]

'Su turkcell durduk yere kac lirami yedi ya kafasina gore paralar cekiyo'

In [16]:
#deasciify tweets dataset
for i in range (len(tweets_df.Tweet)):
    deasciifier = Deasciifier(tweets_df.Tweet[i])
    tweets_df.at[i, 'Tweet'] = deasciifier.convert_to_turkish()

tweets_df.Tweet[17]

'Şu türkcell durduk yere kaç lirami yedi ya kafasına göre paralar çekiyo'

In [17]:
#save the dataframe as csv file
tweets_df.to_csv('tweets_deasciified.csv', index=False)

In [ ]:
#spelling correction
obj = detector.TurkishNLP()
obj.create_word_set()

count = len(tweets_df.Tweet)
for i in range (count):
    lwords = obj.list_words(tweets_df.Tweet[i])
    corrected_words = obj.auto_correct(lwords)
    corrected_string = " ".join(corrected_words)
    tweets_df.at[i, 'Tweet'] = corrected_string
    if(i%100==0):
        print("Processed: ", i, "/", count)

In [19]:
#save the dataframe as csv file
tweets_df.to_csv('tweets_corrected.csv', index=False)

In [21]:
### PART 2 ###
#read tweets dataset - done on deasciified tweets!
tweetspath = '/Users/pinarayaz/Jupyter/NLP/data/tweets_deasciified.csv'
tweets_df = pd.read_csv(tweetspath)
tweets_df.head()

,Tweet,Sentiment
0,Ulan Wifi'ye bağlıyım ben. Ona bağlıyken Türkc...,olumsuz
1,20 dk 1 GB internet 500 mb sadece kaşar türkce...,olumsuz
2,Ayrıca türkcell superonline reklamı kadar da k...,olumsuz
3,Türkcell çok pahalı ya,olumsuz
4,Türkcell Kaş'ta internetin çekmiyor,olumsuz


In [22]:
#apply clean method
count = len(tweets_df.Tweet)
for i in range (count):
    clean_text = clean(tweets_df.Tweet[i])
    tweets_df.at[i, 'Tweet'] = clean_text
tweets_df.head()

,Tweet,Sentiment
0,ulan wifi'ye bağ bağ türkcell ınternet paket b...,olumsuz
1,20 dk 1 gb internet 500 mb kaşar türkcell düş ...,olumsuz
2,türkcell superonline reklam kötü bir reklam gör,olumsuz
3,türkcell paha,olumsuz
4,türkcell kaş'ta internet çek,olumsuz


In [23]:
#save to csv
path = '/Users/pinarayaz/Jupyter/NLP/data/tweets_preprocessed.csv'
tweets_df.to_csv(path, index=False)

## Preprocessing for twt dataset

In [97]:
twtpath = '/Users/pinarayaz/Jupyter/NLP/data/twt.csv'
twt_df = pd.read_csv(twtpath)
twt_df.head()

,Tweet,Label
0,Kız nutellayı o kadar çok abartıyo ki sosyal m...,negative
1,"ooooo dedikodu, alırım bi dal :(",negative
2,keşke :(,negative
3,RT : Bıkmayacaksın :) http://t.co/lmCqMPExbd,positive
4,RT : whatsapp grubuna gelecekler favlasın muha...,positive


In [79]:
#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3', ':))', ':)))'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';(', ':((', ':((('
    ])

#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

In [98]:
#clean with tweet-preprocessor
count = len(twt_df.Tweet)
p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.HASHTAG, p.OPT.RESERVED, p.OPT.NUMBER)
for i in range (count):
    clean_text = p.clean(twt_df.Tweet[i])
    twt_df.at[i, 'Tweet'] = clean_text
twt_df.head()

,Tweet,Label
0,Kız nutellayı o kadar çok abartıyo ki sosyal m...,negative
1,"ooooo dedikodu, alırım bi dal :(",negative
2,keşke :(,negative
3,: Bıkmayacaksın :),positive
4,: whatsapp grubuna gelecekler favlasın muhabbe...,positive


In [106]:
#method to clean the remains
nlp = Turkish()
stopwords = list(STOP_WORDS)

def clean_tweet(tweet):
    #filter using spacy and append it to a string
    tokens = nlp(tweet)
    tokens = [word.lemma_ for word in tokens]
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word not in stopwords and word not in string.punctuation]
    filtered_tweet = []
    for w in tokens:
        if w in emoticons_happy:
            filtered_tweet.append(':)')
        elif w in emoticons_sad:
            filtered_tweet.append(':(')
        else:
            filtered_tweet.append(w)      
    return ' '.join(filtered_tweet)

In [107]:
count = len(twt_df.Tweet)
for i in range (count):
    clean_text = clean_tweet(twt_df.Tweet[i])
    twt_df.at[i, 'Tweet'] = clean_text
twt_df.head()

,Tweet,Label
0,kız nutellayı abartıyo sosyal medya nutella üz...,negative
1,ooooo dedikodu al bi dal :(,negative
2,:(,negative
3,bıkmayacaksın :),positive
4,whatsapp grup gelecek favlasın muhabbet fena :),positive


In [108]:
#save to csv
path = '/Users/pinarayaz/Jupyter/NLP/data/twt_preprocessed.csv'
twt_df.to_csv(path, index=False)

In [ ]:
#deasciify

In [ ]:
#spell correction

## Preprocessing for tremo dataset

In [11]:
#read tremo dataset
tremopath = '/Users/pinarayaz/Jupyter/NLP/data/tremo.csv'
tremo_df = pd.read_csv(tremopath)
tremo_df.head()

,Entry,ValidatedEmotion
0,her yeni gün bir mutluluk,Happy
1,gece kimsenin olmadığı sokaklardan geçerken ço...,Fear
2,gerçekleşemeyen hayaller,Sadness
3,arkadaş kaybetmek beni üzüyor,Sadness
4,insanların çıkarcı olmalarından tiksiniyorum,Disgust


In [14]:
count = len(tremo_df.Entry)
for i in range (count):
    clean_text = clean(tremo_df.Entry[i])
    tremo_df.at[i, 'Entry'] = clean_text
tremo_df.head()

,Entry,ValidatedEmotion
0,yeni gün bir mutlu,Happy
1,gece ol sokak geç kork,Fear
2,gerçek hayal,Sadness
3,arkadaş kaybet üz,Sadness
4,insan çıkar ol tiksin,Disgust


In [15]:
#save to csv
path = '/Users/pinarayaz/Jupyter/NLP/data/tremo_preprocessed.csv'
tremo_df.to_csv(path, index=False)

## Preprocessing for wiki_tr corpus

In [7]:
#read wiki_tr corpus
wikipath = '/Users/pinarayaz/Jupyter/NLP/data/wiki_tr.txt'

with open(wikipath, 'r') as fp:
    lines = fp.readlines()
wiki_df = pd.DataFrame(lines, columns = ['Article'])
wiki_df.head()

,Article
0,cengiz han cenghis khan çinggis haan ya da doğ...
1,film şu anlamlara gelebilir film fotoğrafçılık...
2,mustafa suphidosya manifesto of the communist ...
3,dosya linus torvalds jpeg thumb px linus torva...
4,bolşevik çoğunluktan yana anlamına gelen rusça...


In [9]:
count = len(wiki_df.Article)
for i in range (count):
    clean_text = clean(wiki_df.Article[i])
    wiki_df.at[i, 'Article'] = clean_text
wiki_df.head()

,Article
0,cengiz han cenghis khan çinggis haan doğum ad ...
1,film anlam gel film fotoğrafçılık radyografi s...
2,mustafa suphidosya manifesto of the communist ...
3,dosya linus torvalds jpeg thumb px linus torva...
4,bolşevik çoğunluk yan anlam gel rusça kelime y...


In [10]:
#save to csv
path = '/Users/pinarayaz/Jupyter/NLP/data/wiki_preprocessed.csv'
wiki_df.to_csv(path, index=False)